In [1]:
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import DataCollatorWithPadding
%env TOKENIZERS_PARALLELISM=false

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

env: TOKENIZERS_PARALLELISM=false


In [2]:
def get_logger(filename):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

def preprocess(df):
    df["document_text"] = df["document_text"].astype(str)
    df["document_text"] = df["document_text"].apply(lambda x: " ".join(re.findall(r"[а-яА-Я0-9ёЁ\-\.,?!+a-zA-Z]+", x)))

    return df

In [3]:
def prepare_input(cfg, text):
    inputs = cfg.tokenizer.encode_plus(
        text, 
        return_tensors=None, 
        add_special_tokens=True, 
        max_length=512,
        pad_to_max_length=True,
        truncation=True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['document_text'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        return inputs

In [4]:
def average_pool(last_hidden_states, attention_mask):
    last_hidden = last_hidden_states.masked_fill(
        ~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(
                cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(
                cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()

        self.fc = nn.Linear(self.config.hidden_size, 11)
        self._init_weights(self.fc)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(
                mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(
                mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def feature_me5(self, inputs):
        outputs = self.model(**inputs)
        feature = average_pool(outputs.last_hidden_state,
                               inputs['attention_mask'])
        return feature

    def forward(self, inputs):
        feature = self.feature_me5(inputs)
        output = self.fc(feature)

        return output

In [5]:
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            pred = model(inputs)
                
        preds.append(pred.to('cpu').numpy())
    
    predictions = np.concatenate(preds)
    return predictions

In [6]:
test_path = "test_dataset/dataset.csv"

In [7]:
class CFG:
    num_workers=8
    path="output_intfloat-multilingual-e5-large-extradata_baseline"
    config_path=os.path.join(path, 'config.pth')
    model="intfloat/multilingual-e5-large"
    gradient_checkpointing=False
    batch_size=32
    target_cols=['class_id']
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    max_len=512

CFG.tokenizer = AutoTokenizer.from_pretrained(os.path.join(CFG.path, 'tokenizer'))
LOGGER = get_logger(os.path.join(CFG.path, 'inference'))

In [9]:
# oof_df = pd.read_pickle(os.path.join(CFG.path, 'oof_df.pkl'))
# oof_df.head()
# labels = oof_df[CFG.target_cols].values
# preds = oof_df[[f"pred" for c in CFG.target_cols]].values

# score = f1_score(labels[:, 0], preds[:, 0], average="macro")
# LOGGER.info(f'Score: {score:.4f}')

In [10]:
test = pd.read_csv(test_path, delimiter="|")#.sample(n=1000).reset_index(drop=True)
test = preprocess(test)

if CFG.model == "intfloat/multilingual-e5-large":
    test["document_text"] = test["document_text"].apply(lambda x: "query: " + x)

# test['tokenize_length'] = [len(CFG.tokenizer(text)['input_ids']) for text in test['document_text'].values]
# test = test.sort_values('tokenize_length', ascending=True).reset_index(drop=True)
print(f"test.shape: {test.shape}")
display(test.head())

test.shape: (263, 2)


,document_id,document_text
0,-1704473130726551677,query: Приложение 1 к договору подряда 2 от...
1,-8755312099564304902,query: Директору ООО Вершки-Корешки Головину...
2,4407537979341024747,query: Приложение к договору ренты 15 от 25....
3,-2949797178567945855,query: Приложение 1к договору оказания усл...
4,-3027899216558317249,query: ООО Электросталь Приказ 66 ВО выводе...


In [11]:
test_dataset = TestDataset(CFG, test)
test_loader = DataLoader(test_dataset,
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         collate_fn=DataCollatorWithPadding(tokenizer=CFG.tokenizer, padding='longest'),
                         num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
predictions = []
for fold in CFG.trn_fold:
    model = CustomModel(CFG, config_path=CFG.config_path, pretrained=False)
    state = torch.load(os.path.join(CFG.path, f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth"),
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader, model, device)
    
    predictions.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()

predictions = np.mean(predictions, axis=0)

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

In [12]:
final_labels = [np.argmax(el) for el in predictions]
test["class_id"] = final_labels
test.head()

,document_id,document_text,class_id
0,-1704473130726551677,query: Приложение 1 к договору подряда 2 от...,0
1,-8755312099564304902,query: Директору ООО Вершки-Корешки Головину...,1
2,4407537979341024747,query: Приложение к договору ренты 15 от 25....,0
3,-2949797178567945855,query: Приложение 1к договору оказания усл...,4
4,-3027899216558317249,query: ООО Электросталь Приказ 66 ВО выводе...,8


In [13]:
import pickle

with open ("output_intfloat-multilingual-e5-large-extradata_baseline/executor_le.pkl", "rb") as f:
    exec_le = pickle.load(f)

test.class_id = exec_le.inverse_transform(test.class_id.tolist())
# final_labels_exec = exec_le.inverse_transform(final_labels)
# final_labels_exec[:10]
test.head()

,document_id,document_text,class_id
0,-1704473130726551677,query: Приложение 1 к договору подряда 2 от...,act
1,-8755312099564304902,query: Директору ООО Вершки-Корешки Головину...,application
2,4407537979341024747,query: Приложение к договору ренты 15 от 25....,act
3,-2949797178567945855,query: Приложение 1к договору оказания усл...,contract
4,-3027899216558317249,query: ООО Электросталь Приказ 66 ВО выводе...,order


In [14]:
mapper = {
    "proxy": 1,
    "contract": 2,
    "act": 3,
    "application": 4,
    "order": 5,
    "invoice": 6,
    "bill": 7,
    "arrangement": 8,
    "contract offer": 9,
    "statute": 10,
    "determination": 11,
}

test.class_id = [mapper[el] for el in test.class_id.tolist()]
test.head()
# final_id = [mapper[el] for el in test.class_id.tolist()]
# final_id[:10]

,document_id,document_text,class_id
0,-1704473130726551677,query: Приложение 1 к договору подряда 2 от...,3
1,-8755312099564304902,query: Директору ООО Вершки-Корешки Головину...,4
2,4407537979341024747,query: Приложение к договору ренты 15 от 25....,3
3,-2949797178567945855,query: Приложение 1к договору оказания усл...,2
4,-3027899216558317249,query: ООО Электросталь Приказ 66 ВО выводе...,5


In [23]:
ss = pd.read_csv("test_dataset/submission.csv", delimiter=";")
ss.drop(columns=["class_id"], inplace=True)
print(ss.shape)
ss.head()

(263, 1)


,document_id
0,-6246857835391756489
1,-3556905392020671700
2,7555384868749394362
3,-8330052827067299541
4,-6432527667337110209


In [24]:
ss = ss.merge(test[["document_id", "class_id"]], on="document_id")
print(ss.shape)
ss.head()

(263, 2)


,document_id,class_id
0,-6246857835391756489,3
1,-3556905392020671700,3
2,7555384868749394362,3
3,-8330052827067299541,3
4,-6432527667337110209,3


In [27]:
ss.to_csv("pred_space.csv", index=False, sep=";")

In [26]:
ss.class_id.value_counts()

class_id
3     63
4     63
5     61
2     30
8     11
1     10
11     9
10     9
9      4
7      2
6      1
Name: count, dtype: int64